# Genomic BPE Tests

In [2]:
from pathlib import Path
import collections

from Bio import SeqIO
from fastai.text import *

import sentencepiece as spm

In [3]:
max_sentence_len=20480 #44674217

In [4]:
path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes')
path_to_files = path / 'genome_fastas'

In [ ]:
for i, file in enumerate(path_to_files.iterdir()):
    genome = SeqIO.parse(file, 'fasta')
    chroms = [GB for GB in genome if 'chromosome' in GB.description] #  remove plasmid
    genome = ''.join([i.seq.__str__() for i in chroms]).upper()
    genome += '\n'
    with open('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/sentencepiece_tokenizer/all_files.txt', 'a') as f:
        ind = 0
        while ind < len(genome):
            f.write(genome[ind:ind+max_sentence_len])
            f.write('\n')
            ind += max_sentence_len
    print('completed file {}/{}'.format(i,len(path_to_files.ls())))

In [ ]:
#  changed line in fastai.text see: https://forums.fast.ai/t/multifit-runtime-error-permission-denied/72874/3
#SPProcessor??
#test_proc = SPProcessor(max_vocab_sz=10000)
#ds = TextList.from_folder(path)
#test_proc.train_func
#test_proc.process??
#test_proc.process(TextList.from_folder(path))

In [ ]:
quotemark = '\"'
raw_text_path = '/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/sentencepiece_tokenizer/small_version_all_files.txt'
lang = 'dna'
pre_rules=None
post_rules=None
vocab_sz=None
max_vocab_sz=10000
model_type='bpe'
char_coverage=None
tmp_dir='tmp' 
enc='utf8'
coverage = 0.9998
spec_tokens = ['xxunk', 'xxpad', 'xxbos', 'xxeos'] #['\u2581'+s for s in defaults.text_spec_tok]
cache_dir = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/sentencepiece_tokenizer')/tmp_dir

print(" ".join([
        f"--input={quotemark}{raw_text_path}{quotemark} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={coverage}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f"--model_prefix={cache_dir/'spm'} --vocab_size={30000} --model_type={model_type}"]))

spm.SentencePieceTrainer.Train(" ".join([
        f"--input={quotemark}{raw_text_path}{quotemark} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={coverage}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f"--model_prefix={cache_dir/'spm'} --vocab_size={30000} --model_type={model_type}"]))

In [5]:
s = spm.SentencePieceProcessor(model_file='/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/sentencepiece_tokenizer/tmp/spm.model')

In [6]:
vocabs = [[s.id_to_piece(id), id] for id in range(s.get_piece_size())]

In [15]:
vocabs[-7]

['AATCGCGTTCGGC', 29993]

In [7]:
import random
test_text = "".join([random.choice("ATCG") for _ in range(1000)])

s.encode(test_text, out_type=str, enable_sampling=True, alpha=0.1)

['▁',
 'C',
 'C',
 'AGGA',
 'GAAAT',
 'TACT',
 'GTTC',
 'GCGCAC',
 'GTTC',
 'ATGT',
 'AACCCT',
 'GGCCATC',
 'TCTTCCC',
 'AACTT',
 'GATAAAC',
 'AAGGAC',
 'GAATC',
 'TTTCC',
 'AA',
 'GGCCC',
 'TCCTAC',
 'AAGTT',
 'GTCGT',
 'GATCCC',
 'GAAT',
 'AGAGCC',
 'GAGCGA',
 'C',
 'TT',
 'GGC',
 'AGGAAT',
 'ACTA',
 'G',
 'C',
 'TAC',
 'AAATTTT',
 'AGTGCTCAT',
 'AAAC',
 'GC',
 'AGTC',
 'GGCTAT',
 'TCCCAGT',
 'AATAGA',
 'TAT',
 'ATCC',
 'GGATGC',
 'AGTT',
 'GATTGC',
 'GGAGAC',
 'TGCGTCTT',
 'GAAC',
 'ACAT',
 'GTGCGA',
 'GGCATTC',
 'GGT',
 'TGCTT',
 'GATGT',
 'TACTGC',
 'GACGC',
 'TTTAA',
 'ACAA',
 'GGAC',
 'ACT',
 'ACTCAGA',
 'GTT',
 'GGGCGC',
 'TTAAAT',
 'TTAAT',
 'TGCAC',
 'AGGGC',
 'GCCCTC',
 'ATCT',
 'GA',
 'T',
 'TCCAC',
 'TCCTT',
 'GTTGA',
 'TCCGT',
 'ATAA',
 'GTTATC',
 'AGGTT',
 'AAGT',
 'GCCGT',
 'ACCT',
 'GACTT',
 'GAGACCC',
 'ATTAC',
 'TCC',
 'AATT',
 'AT',
 'GGATGAT',
 'GAGGTC',
 'AGGCCT',
 'GATTC',
 'ACT',
 'AGGCGC',
 'ATGC',
 'AGGA',
 'AT',
 'GC',
 'G',
 'TC',
 'GACAGT',
 'ATT',
 'GATCT'

In [ ]:
#data_lm_spp_fwd = (TextList.from_folder(path, processor=[OpenFileProcessor(), SPProcessor()])
#                  .split_by_rand_pct(0.1, seed=42)
#                  .label_for_lm()
#                  .databunch(bs=128, num_workers=4))
#data_lm_spp_fwd.show_batch()
#parent_dir = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes')
#path_to_files = parent_dir / 'genome_fastas'
#with open(parent_dir / 'all_files_merged.txt', 'w'):
#    pass
#
#files  = set()
#for file in path_to_files.iterdir():
#    files.add(file)
#genome = SeqIO.parse('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/genome_fastas/Corynebacterium striatum.fna', 'fasta')
#chroms = [GB for GB in genome if 'chromosome' in GB.description] #  remove plasmid
#genome = ''.join([i.seq.__str__() for i in chroms]).upper()

In [ ]:
#gen_sp =  SPProcessor()
#data = (TextList.from_folder(parent_dir, processor=[OpenFileProcessor(), gen_sp])
#        .split_by_rand_pct(0.1, seed=42)
#        .label_for_lm()
#        .databunch(bs=128, num_workers=1))

In [ ]:
#x = f.read()
#x = x.replace('\n', '')
#x = x[67:]
#with open('test.txt', 'w') as f:
#    f.write(x)

#import random
#import youtokentome as yttm

#train_data_path = "test.txt"
#model_path = "example.model"

# Generating random file with training data
# 10000 lines with 100 characters in each line
#n_lines = 10000
#n_characters = 100
#with open(train_data_path, "w") as fout:
#    for _ in range(n_lines):
#        print("".join([random.choice("abcd ") for _ in range(n_characters)]), file=fout)


# Training model
#yttm.BPE.train(data=train_data_path, vocab_size=5000, model=model_path)

In [ ]:
# Loading model
#bpe = yttm.BPE(model=model_path)

# Generating random text
#test_text = "".join([random.choice("ATCG") for _ in range(100)])

# Two types of tokenization
#print(bpe.encode([test_text], output_type=yttm.OutputType.ID))
#print(bpe.encode([test_text], output_type=yttm.OutputType.SUBWORD))

In [ ]:
#for i in range(0,100):
#    print(bpe.id_to_subword(i))

In [ ]:
#bpe.vocab_size()